In [1]:
%%capture
%pip install -U transformers
%pip install -U datasets
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U wandb

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("token")
login(token = hf_token)

In [3]:
import pandas as pd
import torch
device = torch.device("cuda")  # для Kaggle

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("token")
login(token = hf_token)

In [4]:
# Загрузка базовой модели с Hugging Face
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

2025-05-14 13:10:32.489905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747228232.663621      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747228232.719546      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",  # Автоматически использует доступный GPU
    torch_dtype=torch.float16,  # Добавьте это
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# Загрузка базовой модели с Hugging Face
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer


In [36]:
from datasets import load_dataset
# Загрузка датасета
dataset = load_dataset("gsm8k", "main", split="train[:100]")  # Берем только 1000 примеров
#dataset = load_dataset("gsm8k", "main", split="train+test")
"""https://huggingface.co/papers/2402.10176
https://arxiv.org/html/2402.10176v1
датасет"""

'https://huggingface.co/papers/2402.10176\nhttps://arxiv.org/html/2402.10176v1\nдатасет'

In [37]:
# Обновите параметры обучения
training_args = TrainingArguments(
    per_device_train_batch_size=8,  # Увеличьте батч
    gradient_accumulation_steps=4,  # Уменьшите аккумуляцию
    gradient_checkpointing=True,
    fp16=True,
    max_steps=500,  # Обучайте по шагам, а не эпохам
    optim="adamw_8bit",  # 8-битный Adam
)

In [38]:

# Функция форматирования промптов
def format_prompt(example):
    return {
        "text": f"Реши математическую задачу:\n{example['question']}\n### Ответ:\n{example['answer']}"
    }

# Применяем форматирование
dataset = dataset.map(
    format_prompt,
    remove_columns=["question", "answer"]
)
# Разделение на трейн/тест (меньший тестовый набор)
dataset = dataset.train_test_split(
    test_size=0.05,
    seed=42
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [39]:
# Настройка LoRA для эффективного дообучения
'''peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj",  # Модули внимания
        "gate_proj", "up_proj", "down_proj"  # Модули FFN
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)'''

# Настройка LoRA - уменьшаем ранг для скорости
peft_config = LoraConfig(
    r=4,                       # Уменьшаем ранг для скорости
    lora_alpha=8,             
    lora_dropout=0.0,         
    bias="none",               
    task_type="CAUSAL_LM"      
)
model = get_peft_model(model, peft_config)


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [40]:
# Настройка обучения
# Ускоренные параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,        # Всего 1 эпоха
    per_device_train_batch_size=16,  # Увеличиваем батч-сайз
    gradient_accumulation_steps=1,  # Уменьшаем аккумуляцию градиентов
    label_names=["labels"],  # Добавьте эту строку
    optim="adamw_torch",
    save_steps=100,            # Реже сохраняем
    logging_steps=5,          # Реже логируем
    learning_rate=5e-4,        # Увеличиваем скорость обучения
    weight_decay=0.0,          # Отключаем регуляризацию
    fp16=True,                
    report_to="none",          # Отключаем отчеты в wandb для скорости
    gradient_checkpointing=False,#it was true
    max_steps=20,             # Ограничиваем количество шагов
    warmup_steps=0,            # Без разогрева
    logging_first_step=True
)

In [41]:
# Настройка тренера (исправленная версия)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    processing_class=tokenizer,  # Заменяем tokenizer на processing_class
    peft_config=peft_config,
)


Converting train dataset to ChatML:   0%|          | 0/95 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/95 [00:00<?, ? examples/s]

In [42]:
# Запускаем обучение
trainer.train()

# Сохраняем модель
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

Step,Training Loss
1,1.768600
5,1.745600
10,1.575800
15,1.449500
20,1.394700


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')